In [55]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from bitstring import BitArray

ALPHA = BitArray("0xc0ac29b7c97c50dd")
BETA = BitArray("0x3f84d5b5b5470917")

class PRINCE:

    def __init__(self, key):
        self.key = key
        self.k0 = key[0:64]
        k0Prime = self.k0.copy()
        k0Prime.ror(1)
        k0Prime ^= self.k0 >> 63
        self.k0Prime = k0Prime
        self.k1 = key[64:128]
        self.RC = (BitArray(hex = '0x0000000000000000'),
                   BitArray(hex = '0x13198a2e03707344'),
                   BitArray(hex = '0xa4093822299f31d0'),
                   BitArray(hex = '0x082efa98ec4e6c89'),
                   BitArray(hex = '0x452821e638d01377'),
                   BitArray(hex = '0xbe5466cf34e90c6c'),
                   BitArray(hex = '0x7ef84f78fd955cb1'),
                   BitArray(hex = '0x85840851f1ac43aa'),
                   BitArray(hex = '0xc882d32f25323c54'),
                   BitArray(hex = '0x64a51195e0e3610d'),
                   BitArray(hex = '0xd3b5a399ca0c2399'),
                   BitArray(hex = '0xc0ac29b7c97c50dd'))
        self.sbox = ('0xb', '0xf', '0x3', '0x2', '0xa', '0xc', '0x9', '0x1',
                     '0x6', '0x7', '0x8', '0x0', '0xe', '0x5', '0xd', '0x4')
        self.invSbox = ('0xb', '0x7', '0x3', '0x2', '0xf', '0xd', '0x8', '0x9',
                        '0xa', '0x6', '0x4', '0x0', '0x5', '0xe', '0xc', '0x1')

    def substitution(self, data, inverse=False):
        ret = BitArray()
        for nibble in data.cut(4):
            if not inverse:
                ret.append(self.sbox[int(nibble.hex, 16)])
            else:
                ret.append(self.invSbox[int(nibble.hex, 16)])
        return ret

    def shiftRows(self, data, inverse=False):
        ret = BitArray(length = 64)
        idx = 0
        for nibble in data.cut(4):
            ret[idx * 4:(idx + 1) * 4] = nibble
            if not inverse:
                idx = (idx + 13) % 16
            else:
                idx = (idx +  5) % 16
        return ret

    def m0(self, data):
        ret = BitArray(length = 16)
        ret[ 0] = data[4] ^ data[ 8] ^ data[12]
        ret[ 1] = data[1] ^ data[ 9] ^ data[13]
        ret[ 2] = data[2] ^ data[ 6] ^ data[14]
        ret[ 3] = data[3] ^ data[ 7] ^ data[11]
        ret[ 4] = data[0] ^ data[ 4] ^ data[ 8]
        ret[ 5] = data[5] ^ data[ 9] ^ data[13]
        ret[ 6] = data[2] ^ data[10] ^ data[14]
        ret[ 7] = data[3] ^ data[ 7] ^ data[15]
        ret[ 8] = data[0] ^ data[ 4] ^ data[12]
        ret[ 9] = data[1] ^ data[ 5] ^ data[ 9]
        ret[10] = data[6] ^ data[10] ^ data[14]
        ret[11] = data[3] ^ data[11] ^ data[15]
        ret[12] = data[0] ^ data[ 8] ^ data[12]
        ret[13] = data[1] ^ data[ 5] ^ data[13]
        ret[14] = data[2] ^ data[ 6] ^ data[10]
        ret[15] = data[7] ^ data[11] ^ data[15]
        return ret

    def m1(self, data):
        ret = BitArray(length = 16)
        ret[ 0] = data[0] ^ data[ 4] ^ data[ 8]
        ret[ 1] = data[5] ^ data[ 9] ^ data[13]
        ret[ 2] = data[2] ^ data[10] ^ data[14]
        ret[ 3] = data[3] ^ data[ 7] ^ data[15]
        ret[ 4] = data[0] ^ data[ 4] ^ data[12]
        ret[ 5] = data[1] ^ data[ 5] ^ data[ 9]
        ret[ 6] = data[6] ^ data[10] ^ data[14]
        ret[ 7] = data[3] ^ data[11] ^ data[15]
        ret[ 8] = data[0] ^ data[ 8] ^ data[12]
        ret[ 9] = data[1] ^ data[ 5] ^ data[13]
        ret[10] = data[2] ^ data[ 6] ^ data[10]
        ret[11] = data[7] ^ data[11] ^ data[15]
        ret[12] = data[4] ^ data[ 8] ^ data[12]
        ret[13] = data[1] ^ data[ 9] ^ data[13]
        ret[14] = data[2] ^ data[ 6] ^ data[14]
        ret[15] = data[3] ^ data[ 7] ^ data[11]
        return ret

    def mPrime(self, data):
        ret = BitArray(length = 64)
        ret[ 0:16] = self.m0(data[ 0:16])
        ret[16:32] = self.m1(data[16:32])
        ret[32:48] = self.m1(data[32:48])
        ret[48:64] = self.m0(data[48:64])
        return ret

    def forwardRound(self, data, idx):
        data = self.substitution(data, inverse=False)
        data = self.mPrime(data)
        data = self.shiftRows(data, inverse=False)
        data ^= self.RC[idx] ^ self.k1
        return data

    def backwardRound(self, data, idx):
        data ^= self.k1 ^ self.RC[idx]
        data = self.shiftRows(data, inverse=True)
        data = self.mPrime(data)
        data = self.substitution(data, inverse=True)
        return data

    def middleRound(self, data):
        data = self.substitution(data, inverse=False)
        data = self.mPrime(data)
        data = self.substitution(data, inverse=True)
        return data

    def princeCore(self, data):
        data ^= self.k1 ^ self.RC[0]
        for idx in (1,2,3,4,5):
            data = self.forwardRound(data, idx)
        data = self.middleRound(data)
        for idx in (6,7,8,9,10):
            data = self.backwardRound(data, idx)
        data ^= self.k1 ^ self.RC[11]
        return data

    def outer(self, data, decrypt=False):
        if decrypt:
            tmp = self.k0
            self.k0 = self.k0Prime
            self.k0Prime = tmp
            self.k1 ^= ALPHA
        data ^= self.k0
        data = self.princeCore(data)
        data ^= self.k0Prime
        if decrypt:
            tmp = self.k0
            self.k0 = self.k0Prime
            self.k0Prime = tmp
            self.k1 ^= ALPHA
        return data

    def encrypt(self, plainText):
        data = self.outer(plainText, decrypt=False)
        return data

    def decrypt(self, cipherText):
        data = self.outer(cipherText, decrypt=True)
        return data

    def printKeys(self):
        print("K0: \t",self.k0)
        print("K1: \t",self.k1)
        print("K0-Prime: \t",self.k0Prime)
        print("K1 ^ K0-Prime: \t",self.k1^self.k0Prime)

In [3]:
!pip install bitstring

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 10.6 MB/s eta 0:00:00


In [56]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import random
from bitstring import BitArray

# Constants used in the PRINCE cipher
ALPHA = BitArray("0xc0ac29b7c97c50dd")
BETA = BitArray("0x3f84d5b5b5470917")

class SlowDiffusionAttack:

    def __init__(self, plainText, key, v2=False):
        """
        Initialize the SlowDiffusionAttack class.

        :param plainText: Input plaintext to be attacked
        :param key: Encryption key for PRINCE or PRINCE_v2
        :param v2: Boolean to indicate if PRINCE_v2 should be used
        """
        self.plainText = plainText


        # Initialize the correct PRINCE object based on the version
        if not self.v2:
            self.princeObj = PRINCE(key)



        # Store generated faulty ciphertexts
        self.cipherTexts = []

    def generate_faulty_cipherTexts(self, reflection=0):
        """
        Generate faulty ciphertexts by injecting faults in specific rounds.

        :param reflection: Indicator to use reflection-based attack
        """
        # Fault injection in specific nibbles (1, 2, 4, 8)
        for faultNibble in (0, 1, 2, 4, 8):
            faultValue = BitArray(f"0x{faultNibble:01x}0000000000000000")

            # Handle no reflection attack
            if not reflection:
                if not self.v2:
                    # Start encryption from the beginning using the PRINCE key and constants
                    data = (self.plainText ^ self.princeObj.k0 ^
                            self.princeObj.k1 ^ self.princeObj.RC[0])


                # Forward rounds (1 to 5)
                for idx in range(1, 6):
                    data = self.princeObj.forwardRound(data, idx)

                # Middle round
                data = self.princeObj.middleRound(data)

                # Backward rounds (6 to 10)
                for idx in range(6, 11):
                    if idx != 8:
                        data = self.princeObj.backwardRound(data, idx)
                    else:
                        # Fault injection happens here
                        if not self.v2:
                            data ^= (self.princeObj.k1 ^
                                     self.princeObj.RC[idx])

                        data ^= faultValue  # Inject fault
                        data = self.princeObj.shiftRows(data, inverse=True)
                        data = self.princeObj.mPrime(data)
                        data = self.princeObj.substitution(data, inverse=True)

                # Final round XORs
                if not self.v2:
                    data ^= (self.princeObj.k0Prime ^
                             self.princeObj.k1 ^ self.princeObj.RC[11])

            else:
                # Reflection-based attack processing
                if not self.v2:
                    # Key inversion and data processing
                    self.princeObj.k1 ^= ALPHA
                    data = (self.plainText ^
                            self.princeObj.k0Prime ^ self.princeObj.k1 ^
                            self.princeObj.RC[0])

                # Proceed with forward rounds and fault injection
                for idx in range(1, 6):
                    if idx != 3:
                        data = self.princeObj.forwardRound(data, idx)
                    else:
                        # Inject fault in round 3
                        data = self.princeObj.substitution(data)
                        data = self.princeObj.mPrime(data)
                        data = self.princeObj.shiftRows(data)
                        data ^= faultValue
                        data ^= self.princeObj.RC[idx] ^ self.princeObj.k1

                # Handle key changes for PRINCE_v2
                if self.v2:
                    self.princeObj.k1 ^= (ALPHA ^ BETA)
                data = self.princeObj.middleRound(data)

                if self.v2:
                    self.princeObj.k0 ^= (ALPHA ^ BETA)
                for idx in range(6, 11):
                    data = self.princeObj.backwardRound(data, idx)

                if not self.v2:
                    data ^= (self.princeObj.k0 ^ self.princeObj.k1 ^
                             self.princeObj.RC[11])
                    self.princeObj.k1 ^= ALPHA
                else:
                    data ^= (self.princeObj.k1 ^ self.princeObj.RC[11])
                    self.princeObj.k0 ^= (BETA ^ ALPHA ^ BETA)
                    self.princeObj.k1 ^= (ALPHA ^ ALPHA ^ BETA)
                    tmp = self.princeObj.k0
                    self.princeObj.k0 = self.princeObj.k1
                    self.princeObj.k1 = tmp

                # Final encryption
                data = self.princeObj.encrypt(data)

            # Store faulty ciphertext
            self.cipherTexts.append(data)

    def launch_attack(self, reflection=0):
        """
        Execute the full Slow Diffusion Attack process.

        :param reflection: Indicates if the reflection-based attack should be used
        """
        print("\n****************************** Launching Slow Diffusion Attack ******************************")
        print("Before Attack: =======================> ")
        self.princeObj.printKeys()

        # Generate faulty ciphertexts with fault injection
        self.generate_faulty_cipherTexts(reflection)

        # Extract key components from faulty ciphertexts
        print("\nExtracting Keys After Fault Injection:")
        k0Prime_xor_k1, K1_0th_col, K1_1st_col, K1_2nd_col, K1_3rd_col = (
            self.check_pattern_state_3(self.cipherTexts)
        )

        # Final check and print results
        if k0Prime_xor_k1:
            print("\nAfter Attack: =======================> ")
            print(f"K0' ⊕ K1: {k0Prime_xor_k1[0].hex}")
        else:
            print("Attack Failed: Key Not Found!")


In [59]:
if __name__ == "__main__":
    from bitstring import BitArray

    # Example plaintext and key for testing
    plaintext =   BitArray("0x0123456789abcdef")
    key = BitArray("0x0123456789abcdeffedcba9876543210")

    print("Starting Slow Diffusion Attack on PRINCE...")

    # Initialize the Slow Diffusion Attack object
    attack = SlowDiffusionAttack(plainText=plaintext, key=key, v2=False)

    # Launch the attack with reflection disabled
    attack.launch_attack(reflection=0)

    print("\nAttack Completed.")


Starting Slow Diffusion Attack on PRINCE...
****************************** Launching Slow Diffusion Attack on PRINCE ******************************
*******************************************************************************************************
Before Attack: =======================> 
K0: 	 0x0123456789abcdef
K1: 	 0xfedcba9876543210
K0-Prime: 	 0x8091a2b3c4d5e6f7
K1 ^ K0-Prime: 	 0x7e4d182bb281d4e7
Faulty Nibble: 	 0 	 (Fixed here)
After Attack: =======================> 
K1: 	 0xfedcba9876543210
K0-Prime: 	 0x8091a2b3c4d5e6f7
K1 ^ K0-Prime: 	 0x7e4d182bb281d4e7

Attack Completed.
